https://ntumlta.github.io/2017fall-ml-hw4/

In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

import os
import pandas as pd

In [22]:
def process_x(x):
    return x.split(' +++$+++ ')[1]

def process_y(x):
    return x.split(' +++$+++ ')[0]

def return_x_y(file_name, dic_size, maxlen):
    train = os.path.join(os.getcwd(),'dataset', file_name)
    df_train = pd.read_csv(train,header=None, delimiter='\n')
    df_train.columns = ['raw']
    df_train['x'] = df_train['raw'].apply(process_x)
    df_train['y'] = df_train['raw'].apply(process_y)
    
    dic_size = dic_size
    docs = df_train['x']
    x_train = [one_hot(d, dic_size) for d in docs]
    
    maxlen = maxlen
    x_train = pad_sequences(x_train, maxlen=maxlen)
    
    y_train = np.array(df_train['y'])
    
    return x_train, y_train


In [23]:
dic_size = 20000
maxlen = 50
# file_name = 'training_label.txt'

x_train, y_train = return_x_y('training_label.txt', dic_size, maxlen)


In [24]:
x_train[:3]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,  1885,  8983,  5087,   351],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0, 16688,  9407, 19959, 14115, 13512,  9407,  7077,
         8492, 15248, 14115, 16649,   290],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,    

In [25]:
ratio_train_valid = 1/4
num_train = int(x_train.shape[0] * (1-ratio_train_valid))
x_train, x_valid = x_train[:num_train], x_train[num_train:]
y_train, y_valid = y_train[:num_train], y_train[num_train:]


In [26]:
x_train.shape

(150000, 50)

In [27]:
model = Sequential()
model.add(Embedding(dic_size, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_valid, y_valid))

Train on 150000 samples, validate on 50000 samples
Epoch 1/1
  8768/150000 [>.............................] - ETA: 7:28 - loss: 0.6009 - acc: 0.6654

KeyboardInterrupt: 

In [ ]:
def process_x_test(x):
    s = x.find(',') + 1
    return x[s:]
    

path_test = os.path.join(os.getcwd(),'dataset', 'testing_data.txt')
df_test = pd.read_csv(path_test,header=0, delimiter='\n')
df_test.columns = ['raw']
df_test['x'] = df_test['raw'].apply(process_x_test)

docs = df_test['x']
x_test = [one_hot(d, dic_size) for d in docs]

x_test = pad_sequences(x_test, maxlen=maxlen)


x_test.shape


In [ ]:

predict = model.predict(x_test[:10], batch_size=32)
predict